## Setup imports

In [1]:
import sys
import os
import logging
import numpy as np
import torch
import monai.networks.nets as nets
from monai.transforms import (
    Compose,
    LoadImaged,
    AddChanneld,
    CropForegroundd,
    ToTensord,
    RandFlipd,
    RandAffined,
    SpatialPadd,
    Activationsd,
    Activations,
    Resized,
    AsDiscreted,
    AsDiscrete,
    GaussianSmoothd,
    SpatialCropd,
)
from transforms import (
    CTWindowd,
    CTSegmentation,
    RelativeCropZd,
    RandGaussianNoised,
)
from monai.data import DataLoader, Dataset, PersistentDataset, CacheDataset
from torchsampler import ImbalancedDatasetSampler
from monai.transforms.croppad.batch import PadListDataCollate
from monai.utils import NumpyPadMode, set_determinism
from monai.utils.enums import Method
from monai.config import print_config
from sklearn.model_selection import train_test_split
from trainer import Trainer
from validator import Validator
from tester import Tester
from utils import (
    multi_slice_viewer,
    setup_directories,
    get_data_from_info,
    large_image_splitter,
    calculate_class_imbalance,
    create_device,
    balance_training_data,
    balance_training_data2,
    transform_and_copy,
    convert_labels,
    load_mixed_images,
)
from test_data_loader import TestDataset
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()

MONAI version: 0.5.dev2115
Numpy version: 1.20.2
Pytorch version: 1.8.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: c9990b040832792aa897b27a07180d4629cb8de3

Optional dependencies:
Pytorch Ignite version: 0.4.4
Nibabel version: 3.2.1
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 8.1.2
Tensorboard version: 2.4.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.9.0+cu111
ITK version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.59.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.8.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Setup directories

In [2]:
dirs = setup_directories()

## Setup torch device

In [3]:
# pass "cuda" to use the GPU
device, using_gpu = create_device("cuda")

## Load and randomize images

In [4]:
# HACKATON image and segmentation data
hackathon_dir = os.path.join(dirs["data"], 'HACKATHON')
map_fn = lambda x: (x[0], int(x[1]))
with open(os.path.join(hackathon_dir, "train.txt"), 'r') as fp:
    train_info_hackathon = [map_fn(entry.strip().split(',')) for entry in fp.readlines()]
image_dir = os.path.join(hackathon_dir, 'images', 'train')
seg_dir = os.path.join(hackathon_dir, 'segmentations', 'train')
_train_data_hackathon = get_data_from_info(image_dir, seg_dir, train_info_hackathon)
_train_data_hackathon = large_image_splitter(_train_data_hackathon, dirs["cache"], 4, only_label_one=True)
mixed_images = load_mixed_images(dirs["data"])
_train_data_hackathon.extend(mixed_images)
#copy_list = transform_and_copy(_train_data_hackathon, dirs['cache'])
#balance_training_data2(_train_data_hackathon, copy_list, seed=72)

# One channel
convert_labels(_train_data_hackathon, dtype=np.float32, as_array=True)
# Two channel
#convert_labels(_train_data_hackathon, dtype=np.int64, as_array=False)
    
# PSUF data
"""psuf_dir = os.path.join(dirs["data"], 'psuf')
with open(os.path.join(psuf_dir, "train.txt"), 'r') as fp:
    train_info = [entry.strip().split(',') for entry in fp.readlines()]
image_dir = os.path.join(psuf_dir, 'images')
train_data_psuf = get_data_from_info(image_dir, None, train_info)"""
# Split data into train, validate and test
train_split, test_data_hackathon = train_test_split(_train_data_hackathon, test_size=0.2, shuffle=True)#, random_state=42)
#copy_list = transform_and_copy(train_split, dirs['cache'])
#balance_training_data2(train_split, copy_list, seed=72)
train_data_hackathon, valid_data_hackathon = train_test_split(train_split, test_size=0.2, shuffle=True)#, random_state=43)

#convert_labels(train_data_hackathon, dtype=np.int64, as_array=False)
#convert_labels(valid_data_hackathon, dtype=np.int64, as_array=False)
#convert_labels(test_data_hackathon, dtype=np.int64, as_array=False)

#balance_training_data(train_data_hackathon, seed=72)
#balance_training_data(valid_data_hackathon, seed=73)
#balance_training_data(test_data_hackathon, seed=74)

Splitting large images...
original data len: 355
new data len: 396


## Setup transforms

In [5]:
# Crop foreground
crop_foreground = CropForegroundd(
    keys=["image"],
    source_key="image",
    margin=(5, 5, 0),
    select_fn = lambda x: x != 0
)
# Crop Z
crop_z = RelativeCropZd(keys=["image"], relative_z_roi=(0.05, 0.15))

# Window width and level (window center)
WW, WL = 1500, -600
ct_window = CTWindowd(keys=["image"], width=WW, level=WL)
# Random flip axis
rand_x_flip = RandFlipd(keys=["image"], spatial_axis=0, prob=0.50)
rand_y_flip = RandFlipd(keys=["image"], spatial_axis=1, prob=0.50)
rand_z_flip = RandFlipd(keys=["image"], spatial_axis=2, prob=0.50)
# Rand affine transform
rand_affine = RandAffined(
    keys=["image"],
    prob=0.50,
    rotate_range=(0, 0, np.pi/12),
    shear_range=(0.07, 0.07, 0.0),
    translate_range=(0, 0, 0),
    scale_range=(0.07, 0.07, 0.0),
    padding_mode="zeros"
)
# Pad image to have hight at least 30
spatial_pad = SpatialPadd(keys=["image"], spatial_size=(-1, -1, 30))
# Resize image x and y
resize_fator = 0.50
xy_size = int(512*resize_fator)
#resize = Resized(keys=["image"], spatial_size=(int(512*resize_fator), int(512*resize_fator), -1), mode="trilinear")
resize1 = Resized(keys=["image"], spatial_size=(-1, -1, 40), mode="area")
resize2 = Resized(keys=["image"], spatial_size=(xy_size, xy_size, -1), mode="area")
# spatioal crop
crop = SpatialCropd(keys=["image"], roi_start=(0, 0, 4), roi_end=(xy_size, xy_size, 36))
# Apply Gaussian noise
rand_gaussian_noise = RandGaussianNoised(keys=["image"], prob=0.25, mean=0.0, std=0.05)
# gaussian smooth
gaussian_noise_smooth = GaussianSmoothd(keys=["image"], sigma=(0.2, 0.2, 0.0))

#### Create transforms

In [6]:
common_transform = Compose([
    LoadImaged(keys=["image"]),
    ct_window,
    CTSegmentation(keys=["image"]),
    AddChanneld(keys=["image"]),
    crop_foreground,
    #crop_z,
    gaussian_noise_smooth,
    resize1,
    resize2,
    crop,
])
hackathon_train_transform = Compose([
    common_transform,
    rand_x_flip,
    rand_y_flip,
    rand_z_flip,
    rand_affine,
    rand_gaussian_noise,
    ToTensord(keys=["image"]),
]).flatten()
hackathon_valid_transfrom = Compose([
    common_transform,
    ToTensord(keys=["image"]),
]).flatten()
hackathon_test_transfrom = Compose([
    common_transform,
    ToTensord(keys=["image"]),
]).flatten()
psuf_transforms = Compose([
    LoadImaged(keys=["image"]),
    AddChanneld(keys=["image"]),
    ToTensord(keys=["image"]),
])

## Setup data

In [7]:
#set_determinism(seed=100)
train_dataset = PersistentDataset(data=train_data_hackathon[:], transform=hackathon_train_transform, cache_dir=dirs["persistent"])
valid_dataset = PersistentDataset(data=valid_data_hackathon[:], transform=hackathon_valid_transfrom, cache_dir=dirs["persistent"])
test_dataset = PersistentDataset(data=test_data_hackathon[:], transform=hackathon_test_transfrom, cache_dir=dirs["persistent"])
_, n, p = calculate_class_imbalance(train_data_hackathon)
train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    pin_memory=using_gpu,
    num_workers=2,
    #sampler=ImbalancedDatasetSampler(train_data_hackathon, num_samples=2*p, callback_get_label=lambda x, i: x[i]['_label']),
    collate_fn=PadListDataCollate(Method.SYMMETRIC, NumpyPadMode.CONSTANT)
)
_, n, p = calculate_class_imbalance(valid_data_hackathon)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=4,
    #shuffle=True,
    pin_memory=using_gpu,
    num_workers=2,
    #sampler=ImbalancedDatasetSampler(valid_data_hackathon, num_samples=2*p, callback_get_label=lambda x, i: x[i]['_label']),
    collate_fn=PadListDataCollate(Method.SYMMETRIC, NumpyPadMode.CONSTANT)
)
test_loader = DataLoader(
    test_dataset,
    batch_size=4,
    #shuffle=True,
    pin_memory=using_gpu,
    num_workers=2,
    collate_fn=PadListDataCollate(Method.SYMMETRIC, NumpyPadMode.CONSTANT)
)

## Setup network, loss function, optimizer and scheduler

In [8]:
out_channels = 1
#network = nets.EfficientNetBN("efficientnet-b4", spatial_dims=3, in_channels=1, num_classes=out_channels).to(device)
network = nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=out_channels).to(device)
# pos_weight for class imbalance
_, n, p = calculate_class_imbalance(train_data_hackathon)
print(n, p)
##############################################################
# One channel
if out_channels == 1:
    pos_weight = torch.Tensor([n/p]).to(device)
    loss_function = torch.nn.BCEWithLogitsLoss(pos_weight)
# Two channel
else:
    pos_weight = torch.Tensor([n, p]).to(device)
    loss_function = torch.nn.CrossEntropyLoss(pos_weight)
##############################################################
optimizer = torch.optim.Adam(network.parameters(), lr=1e-3, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95, last_epoch=-1)

214 64


## Setup validator and trainer

In [9]:
# Setup validator and trainer
valid_post_transforms = Compose([
    Activationsd(keys="pred", sigmoid=True), # One channel
    #Activationsd(keys="pred", softmax=True),  # Two channel
])
validator = Validator(
    device=device,
    val_data_loader=valid_loader,
    network=network,
    loss_function=loss_function,
    post_transform=valid_post_transforms,
    n_classes=out_channels,
    patience=30,
    amp=using_gpu,
    non_blocking=using_gpu
)

trainer = Trainer(
    device=device,
    out_dir=dirs["out"],
    out_name="DenseNet121_1",
    max_epochs=60,
    validation_epoch = 1,
    validation_interval = 1,
    train_data_loader=train_loader,
    network=network,
    optimizer=optimizer,
    loss_function=loss_function,
    lr_scheduler=scheduler,
    validator=validator,
    amp=using_gpu,
    non_blocking=using_gpu
)
validator.early_stop_handler.set_trainer(trainer)

## Run trainer

In [10]:
train_output = trainer.run()

Training started: 17/04/2021 15:35:18
INFO:ignite.engine.engine.Trainer:Engine run resuming from iteration 0, epoch 0 until 60 epochs
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 1/70 -- loss: 2.6527 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 2/70 -- loss: 2.2384 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 3/70 -- loss: 3.7837 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 4/70 -- loss: 3.1593 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 5/70 -- loss: 2.2770 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 6/70 -- loss: 1.7018 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 7/70 -- loss: 1.2207 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 8/70 -- loss: 0.9318 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 9/70 -- loss: 1.3176 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 10/70 -- loss: 0.8395 
INFO:ignite.engine.engine.Trainer:Epoch: 1/60, Iter: 11/70 -- loss: 2.7955 
INFO:ignite.engine.engine.Trainer:Epoch

/opt/conda/lib/python3.7/site-packages/monai/handlers/stats_handler.py:196: UserWarning: ignoring non-scalar output in StatsHandler, make sure `output_transform(engine.state.output)` returns a scalar or dictionary of key and scalar pairs to avoid this warning. image:<class 'torch.Tensor'>
  " {}:{}".format(name, type(value))
/opt/conda/lib/python3.7/site-packages/monai/handlers/stats_handler.py:196: UserWarning: ignoring non-scalar output in StatsHandler, make sure `output_transform(engine.state.output)` returns a scalar or dictionary of key and scalar pairs to avoid this warning. label:<class 'torch.Tensor'>
  " {}:{}".format(name, type(value))
/opt/conda/lib/python3.7/site-packages/monai/handlers/stats_handler.py:196: UserWarning: ignoring non-scalar output in StatsHandler, make sure `output_transform(engine.state.output)` returns a scalar or dictionary of key and scalar pairs to avoid this warning. pred:<class 'torch.Tensor'>
  " {}:{}".format(name, type(value))
/opt/conda/lib/pytho

INFO:ignite.engine.engine.Validator:Got new best metric of Valid_AUC: 0.906
INFO:ignite.engine.engine.Validator:Epoch[1] Metrics -- Valid_ACC: 0.8000 Valid_AUC: 0.9060 Valid_Loss: 2.2698 
INFO:ignite.engine.engine.Validator:Key metric: Valid_AUC best value: 0.906 at epoch: 1
INFO:ignite.engine.engine.Validator:Epoch[1] Complete. Time taken: 00:00:51
INFO:ignite.engine.engine.Validator:Engine run complete. Time taken: 00:00:51
INFO:ignite.engine.engine.Trainer:Saved checkpoint at epoch: 1
INFO:ignite.engine.engine.Trainer:Epoch[1] Complete. Time taken: 00:04:43
INFO:ignite.engine.engine.Trainer:Epoch: 2/60, Iter: 1/70 -- loss: 3.6405 
INFO:ignite.engine.engine.Trainer:Epoch: 2/60, Iter: 2/70 -- loss: 2.3567 
INFO:ignite.engine.engine.Trainer:Epoch: 2/60, Iter: 3/70 -- loss: 2.2472 
INFO:ignite.engine.engine.Trainer:Epoch: 2/60, Iter: 4/70 -- loss: 1.8497 
INFO:ignite.engine.engine.Trainer:Epoch: 2/60, Iter: 5/70 -- loss: 0.6621 
INFO:ignite.engine.engine.Trainer:Epoch: 2/60, Iter: 6/70 

## Setup tester

In [11]:
tester = Tester(
    device=device,
    test_data_loader=test_loader,
    load_dir=train_output,
    out_dir=dirs["out"],
    network=network,
    n_classes=out_channels,
    post_transform=valid_post_transforms,
    non_blocking=using_gpu,
    amp=using_gpu
)

## Run tester

In [12]:
tester.run()

INFO:ignite.engine.engine.Tester:Engine run resuming from iteration 0, epoch 0 until 1 epochs
INFO:ignite.engine.engine.Tester:Restored all variables from /home/jupyter/RIS/out/training/DenseNet121_1_2021-04-17_15-35-18/network_key_metric=0.9850.pt
INFO:ignite.engine.engine.Tester:Got new best metric of Test_AUC: 0.9927849927849928
INFO:ignite.engine.engine.Tester:Epoch[1] Metrics -- Test_ACC: 0.9655 Test_AUC: 0.9928 
INFO:ignite.engine.engine.Tester:Key metric: Test_AUC best value: 0.9927849927849928 at epoch: 1
INFO:ignite.engine.engine.Tester:Epoch[1] Complete. Time taken: 00:01:01
INFO:ignite.engine.engine.Tester:Engine run complete. Time taken: 00:01:02


In [13]:
act = Activations(sigmoid=True) # One channel
#act = Activations(softmax=True)  # Two channel
d = AsDiscrete(threshold_values=True)
network.eval()
with torch.no_grad():
    for test_data in test_loader:
        test_images, test_labels = test_data["image"].to(device), test_data["label"].to(device)
        test_outputs = act(network(test_images))
        _test_outputs = d(test_outputs)
        print([x.item() for x in test_outputs.detach().cpu().numpy()])
        print([x.item() for x in _test_outputs.detach().cpu().numpy()])
        print([x.item() for x in test_labels.detach().cpu().numpy()])
        print()
        #print(np.array([test_outputs.detach().cpu().numpy(), _test_outputs.detach().cpu().numpy(), test_labels.detach().cpu().numpy()]))

[0.13651688396930695, 0.7572658658027649, 0.005492628552019596, 0.007428611628711224]
[0.0, 1.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0]

[0.781683623790741, 0.6961158514022827, 0.018689649179577827, 0.013438568450510502]
[1.0, 1.0, 0.0, 0.0]
[1.0, 1.0, 0.0, 0.0]

[0.06857097148895264, 0.007614025846123695, 0.24822117388248444, 0.009975560940802097]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]

[0.7129674553871155, 0.015824977308511734, 0.0251078549772501, 0.08828529715538025]
[1.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0]

[0.8009762167930603, 0.012599634937942028, 0.01577937602996826, 0.2788048982620239]
[1.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0]

[0.0707487091422081, 0.7917895317077637, 0.08367335051298141, 0.9036087989807129]
[0.0, 1.0, 0.0, 1.0]
[0.0, 1.0, 0.0, 1.0]

[0.25284337997436523, 0.02369968593120575, 0.5813652276992798, 0.010347261093556881]
[0.0, 0.0, 1.0, 0.0]
[0.0, 0.0, 1.0, 0.0]

[0.44700679183006287, 0.017399558797478676, 0.010715628042817116, 0.00735123548656702]
[0.0, 0.0, 0.0, 0.0]

## Plot results

In [14]:
%matplotlib widget
import matplotlib.pyplot as plt

#### Loss

In [15]:
train_loss = np.hsplit(np.loadtxt(os.path.join(train_output, 'log_loss.txt')), 2)
valid_loss = np.hsplit(np.loadtxt(os.path.join(train_output, 'log_Valid_Loss.txt')), 2)

In [16]:
fig, ax = plt.subplots()
ax.plot(train_loss[0], train_loss[1], valid_loss[0], valid_loss[1])
ax.set(xlabel='interation', ylabel='loss',
       title='Loss')
ax.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### AUC and ACC

In [17]:
valid_auc = np.hsplit(np.loadtxt(os.path.join(train_output, 'log_Valid_AUC.txt')), 2)
valid_acc = np.hsplit(np.loadtxt(os.path.join(train_output, 'log_Valid_ACC.txt')), 2)

In [18]:
fig, ax = plt.subplots()
ax.plot(valid_auc[0], valid_auc[1], valid_acc[0], valid_acc[1])
ax.set(xlabel='interation', ylabel='AUC and ACC',
       title='AUC and ACC')
ax.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …